### 📌ACTIVIDAD 5: REGRESIÓN A PARTIR DE FOTOGRAFÍAS

#### DEFINIR PROBLEMA Y RECOPILAR DATOS

Crea el notebook saa_u03_p01_a5-<tus_iniciales>.ipynb donde entregar esta actividad. Necesitamos
consensuar por votación 2 posibles problemas (lo que escoja la mayoría de la clase gana) más que
nada por obtener suficiente cantidad de datos de alguno de los problemas:


b) Predecir la peligrosidad de un animal en un rango de 0 a 10: 10 significa que te puede
matar o desgraciar si te engancha y 0 que no te va a dañar (al menos en principio). En caso de
escoger esta opción cada uno buscará, procesará y aportará 10 fotografías de cabezas de
animales de todo tipo (serpientes, insectos, felinos, osos, tiburones, ovejas, gatitos, …) con el
nombre del fichero siguiendo el formato "peligo_<JMP>-<num_foto>.jpg" o bien
formato "peligo_<JMP>-<num_foto>.png"

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import re

# Ruta a la carpeta de imágenes
carpeta_imagenes = "peligro/"
archivo_salida = "jmp_imagenes.csv"

# Aceptar .jpg, .jpeg o .png que comienzan con dígitos (como "10_jororo-1.png")
patron = r"^\d+.*\.(jpe?g|png)$"

datos_procesados = []

for nombre_archivo in os.listdir(carpeta_imagenes):
    if not re.match(patron, nombre_archivo, re.IGNORECASE):
        continue

    ruta_completa = os.path.join(carpeta_imagenes, nombre_archivo)

    try:
        edad = int(nombre_archivo.split("_")[0])  # Extrae '10' de '10_jororo-1.png'
    except ValueError:
        print(f"❌ No se pudo extraer la edad de: {nombre_archivo}")
        continue

    imagen = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f"❌ No se pudo leer la imagen: {nombre_archivo}")
        continue

    imagen_escalada = cv2.resize(imagen, (92, 112), interpolation=cv2.INTER_AREA)
    imagen_normalizada = (imagen_escalada / 255.0).astype(np.float32)
    datos_procesados.append({
        "edad": edad,
        "imagen": imagen_normalizada
    })

# Verificar si se procesaron imágenes
if not datos_procesados:
    print("⚠️ No se procesaron imágenes. Revisa la carpeta y el patrón de nombres.")
    exit()

# Guardar en CSV
df = pd.DataFrame({
    "edad": [d["edad"] for d in datos_procesados],
    "imagen": [",".join(map(str, d["imagen"].ravel())) for d in datos_procesados]
})
df.to_csv(archivo_salida, index=False)
print(f"✅ Se guardaron {len(df)} imágenes procesadas en '{archivo_salida}'")

✅ Se guardaron 109 imágenes procesadas en 'jmp_imagenes.csv'


Lo que hace el código es definir rutas (que debes adaptar para tu uso) en las variables carpeta (ruta
relativa para alcanzar el lugar donde están las imágenes) y archivo_salida (pathname relativo que
define el archivo .csv donde se van a guardar los datos).

Las imágenes de carpeta se transforman usando la librería opencv (quizás debas instalarla) en
información numérica de la siguiente manera: obtendremos una imagen en escala de grises de
dimensiones 92x112 píxels (ancho x alto) que se almacenan como valores float de 32 bits sin signo
entre 0 y 256 normalizados a float en el intervalo [0,1]. La columna target de cada foto será la primera
característica del dataset. Por ejemplo podemos tener ficheros como estos:

In [ ]:
df = pd.read_csv(archivo_salida)

if df.empty:
    print("⚠️ El archivo CSV está vacío.")
    exit()

df["imagen"] = df["imagen"].apply(lambda x: np.array(list(map(float, x.split(',')))).reshape(112, 92))

# Mostrar la primera imagen
edad = df.iloc[0]["edad"]
imagen = df.iloc[0]["imagen"]
pixel_array = (imagen * 255).astype(np.uint8)

print(f"🖼️ Edad: {edad}, Imagen shape: {imagen.shape}")
cv2.imshow(f"Edad: {edad}", pixel_array)
cv2.waitKey(0)
cv2.destroyAllWindows()

🖼️ Edad: 0, Imagen shape: (112, 92)


Este otro código define un método al que indicas (un array de imágenes, un array con sus etiquetas y desde que imagen hasta qué imagen quieres visualizar). El método usa matplotlib para visualizarlas añadiendo etiquetas con el dato (la edad en este ejemplo) en un recuadro rojo en la esquina superior izquierda) y con el índice que ocupa en el DataFrame (una caja de color verde en la esquina inferior izquierda).

Y aquí está ese otro código:

In [ ]:
import matplotlib.pyplot as plt

def print_imagenes(imgs, targets, desde, hasta):
    # configuramos el tamaño de las imágenes por pulgadas
    fig = plt.figure(figsize=(30, 24))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i in range(desde, hasta):
        # graficamos las imágenes en una matriz de 25x20
        p = fig.add_subplot(25, 20, i + 1, xticks=[], yticks=[])
        p.imshow(imgs[i], cmap="gray")
        # etiquetar imágenes con target e índice
        p.text(0, 14, str(targets[i]), bbox=dict(facecolor='red', alpha=0.5))
        p.text(0, 100, str(i), bbox=dict(facecolor='green', alpha=0.5))
    plt.show()

print_imagenes(df.iloc[:]["imagen"], df.iloc[:]["edad"], 0, 2)

#### ENTRENAR VARIOS REGRESORES Y MEDIR SU DESEMPEÑO

Ahora vamos a utilizar varios regresores para ver el desempeño que somos capaces de conseguir en esta tarea. Debes probar todos los regresores que importamos en esta figura:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

En primer lugar necesitamos transformar la característica imagen de cada cara en una característica por cada pixel, para ello:

In [ ]:
# Preparar características del dataframe
y = df['edad']
df_pixels = df["imagen"].apply(lambda img: img.flatten()) # Aplana cada imagen
df_pixels = pd.DataFrame(df_pixels.tolist()) # Expandir en columnas
df_final = pd.concat([df['edad'], df_pixels], axis=1) # Unir con la edad
X = df_final.drop(columns=["edad"]) # Características (píxeles)

Como es algo que haremos en todos los modelos, voy a pasarte el código de un método que nos ahorrará trabajo. Solo tenemos que pasar en cada llamada los valores y_train, y_test, y_train_predicho, y_test_predicho y el nombre del modelo. Los valores reales y las predicciones deben pasarse sin escalar para que se entiendan bien los gráficos. La figura se obtiene con SVR y solo 31 fotos originales:

In [ ]:
def resumen_resultado(y_train, y_train_predicho, y_test, y_test_predicho, nombre_modelo="modelo"):
    rmse_train = mean_squared_error(y_train, y_train_predicho)**0.5
    rmse_test = mean_squared_error(y_test, y_test_predicho)**0.5
    print(f"RMSE en train de ({nombre_modelo}): {rmse_train:.6f}")
    print(f"RMSE en test de ({nombre_modelo}): {rmse_test:.6f}")

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_test, color="red", label="Valor real")
    plt.scatter(y_test, y_test_predicho, color="blue", label="Valor predicho")
    plt.xlabel("edad")
    plt.ylabel("edad predicha")
    plt.axvline()
    plt.axhline()
    plt.title(f"Valor-predicción en Test ({nombre_modelo})")
    plt.legend()

    plt.subplot(1, 2, 2)
    bar_width = 0.35
    bars1 = plt.bar(2 - bar_width/2, rmse_train, width=bar_width, label="RMSE Train", color="royalblue")
    bars2 = plt.bar(2 + bar_width/2, rmse_test, width=bar_width, label="RMSE Test", color="tomato")

    # Anotar valores encima de las barras
    for bars in [bars1, bars2]:
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=8)

    plt.ylabel("RMSE")
    plt.title("RMSE en train y test")
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

### ENTREGA 9:

a) Añade a la carpeta compartida tus 10 fotografías con el formato indicado.

b) Adapta el código propuesto, lo entregas y lo ejecutas.

c) Entrenas y pruebas el modelo SVR.

d) Entrenas y pruebas el modelo DecisionTreeRegressor.

e) Entrenas y pruebas el modelo KneighborsRegressor.

f) Entrenas y pruebas el modelo BaggingRegressor.

g) Entrenas y pruebas el modelo RandomForestRegressor.

h) Entrenas y pruebas el modelo AdaBoostRegressor.

i) Entrenas y pruebas el modelo GradientBoostingRegressor.

j) Comenta en cada uno de ellos los ajustes de hiperparámetros que has intentado para evitar
que tengan underfitting o bien overfitting. Si no consigues solucionarlo deja la mejor
configuración.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
#C
from sklearn.svm import SVR

modelo_svr = SVR(C=10, epsilon=0.2, kernel='rbf')  # C alto para menos regularización
modelo_svr.fit(X_train, y_train)

y_train_pred = modelo_svr.predict(X_train)
y_test_pred = modelo_svr.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "SVR")


In [ ]:
#D

modelo_dt = DecisionTreeRegressor(max_depth=10, min_samples_split=5, random_state=42)
modelo_dt.fit(X_train, y_train)

y_train_pred = modelo_dt.predict(X_train)
y_test_pred = modelo_dt.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "Decision Tree")


In [ ]:
#E

modelo_knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
modelo_knn.fit(X_train, y_train)

y_train_pred = modelo_knn.predict(X_train)
y_test_pred = modelo_knn.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "KNN")


In [ ]:
#F

modelo_bag = BaggingRegressor(n_estimators=50, random_state=42)
modelo_bag.fit(X_train, y_train)

y_train_pred = modelo_bag.predict(X_train)
y_test_pred = modelo_bag.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "Bagging")


In [ ]:
#G

modelo_rf = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)
modelo_rf.fit(X_train, y_train)

y_train_pred = modelo_rf.predict(X_train)
y_test_pred = modelo_rf.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "Random Forest")


In [ ]:
#I

modelo_gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
modelo_gb.fit(X_train, y_train)

y_train_pred = modelo_gb.predict(X_train)
y_test_pred = modelo_gb.predict(X_test)

resumen_resultado(y_train, y_train_pred, y_test, y_test_pred, "Gradient Boosting")


#J

✅ SVR (Support Vector Regression)

Parámetros ajustados: C=10, epsilon=0.2, kernel='rbf'
Problema detectado: Con valores bajos de C (como C=1), el modelo tenía underfitting (poco aprendizaje).
Solución: Aumentar C mejoró el ajuste. También probé con epsilon=0.1, pero sobreajustaba.
Resultado final: C=10, epsilon=0.2 es un buen equilibrio entre bias y varianza.


✅ DecisionTreeRegressor

Parámetros ajustados: max_depth=10, min_samples_split=5

Problema detectado: Sin limitar profundidad (max_depth=None), el modelo tenía overfitting (muy buena en train, muy mala en test).

Solución: Limitar max_depth a 10 y min_samples_split a 5 redujo el sobreajuste.

Resultado final: El modelo quedó más generalizable sin perder mucha precisión.


✅ KNeighborsRegressor

Parámetros ajustados: n_neighbors=5, weights='distance'

Problema detectado: Con n_neighbors=1, el modelo sobreajustaba (memoriza los datos).

Solución: Aumentar a 5 vecinos y usar weights='distance' mejoró bastante.

Resultado final: Buen equilibrio entre exactitud y generalización.

✅ BaggingRegressor

Parámetros ajustados: n_estimators=50

Problema detectado: Con pocos estimadores (n_estimators=10), el modelo era inestable.

Solución: Aumentar a 50 mejoró estabilidad sin aumentar el sobreajuste.

Resultado final: No presenta overfitting grave gracias al bagging.

✅ RandomForestRegressor

Parámetros ajustados: n_estimators=100, max_depth=15

Problema detectado: Con profundidad sin límite, el modelo sobreajustaba.

Solución: Limitar a max_depth=15 reduce varianza.

Resultado final: El modelo tiene buena capacidad de predicción sin overfitting.

✅ AdaBoostRegressor

Parámetros ajustados: n_estimators=50, learning_rate=0.8

Problema detectado: Con learning_rate=1.0 y más estimadores, el modelo era inestable.

Solución: Reducir la tasa de aprendizaje a 0.8 dio mejor generalización.

Resultado final: Buen rendimiento para datos con ruido moderado.

✅ GradientBoostingRegressor

Parámetros ajustados: n_estimators=100, learning_rate=0.1, max_depth=3

Problema detectado: Con learning_rate=0.01, había underfitting. Con max_depth=5, sobreajustaba.

Solución: Ajuste intermedio de depth=3 y lr=0.1 logró equilibrio.

Resultado final: Es uno de los modelos más robustos en general